In [63]:
import pandas as pd
from sodapy import Socrata

def raw_data_socrata(limit=10):
    client = Socrata("data.cityofnewyork.us", None)
    field_list = ','.join(['unique_key','created_date','borough','incident_zip','city','complaint_type'])
    results = client.get("fhrw-4uyv",limit,select=field_list,where="created_date='2017'")
    df = pd.DataFrame.from_records(results)
    return df

def raw_data_local():
    # Reads in dataframe from local 2017 subset
    df = pd.read_csv('./2017_subset.csv',dtype='object')
    return df

def zip_to_borough_read():
    zip_to_borough = pd.read_csv('./zip_to_borough.csv',dtype='object')
    return zip_to_borough

def us_zip_populations_read():
    # Reads in the ZCTA file with necessary datatypes and names for easy merging
    us_zip_populations = pd.read_csv('./ZCTA.csv',dtype={0:'object',1:'int64'})
    us_zip_populations.columns = ['ZIPCODE','POPULATION']
    return us_zip_populations

def cleaned_data(df):
    # Reads in zip<>borough dimension table for nyc area
    zip_to_borough = zip_to_borough_read()
    # Joins in second borough column based on zip, to correct for Unspecified ones
    df = df.merge(zip_to_borough,on='incident_zip',how='outer')
    # Only leaves numeric characters in zipcode field
    df['incident_zip'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
    # Only leaves first five digits to fix overly long zips ex: 10456-1023
    df['incident_zip'] = df['incident_zip'].str[:5]
    return df
    
def borough_complaint_stats(df):
    #Consider only the 10 most common overall complaint types. 
    #For each borough, how many of each of those 10 types were there in 2017?
    s1 = df['complaint_type'].groupby(df['BOROUGH']).value_counts()
    top10_by_borough = s1.groupby(level=[0]).nlargest(10)
    return top10_by_borough

def zip_complaint_stats(df):
    #Consider only the 10 most common overall complaint types.
    #For the 10 most populous zip codes, how many of each of those 10 types were there in 2017?
    # Set of unique zips from raw the data in it's own dataframe
    nyc_zips = pd.Series(df['incident_zip'].unique()).to_frame(name='ZIPCODE')
    # Generates 10 top zipcodes in NYC with their populations
    nyc_zip_populations = nyc_zips.merge(us_zip_populations_read(),on='ZIPCODE')
    top_zip_populations = nyc_zip_populations.sort_values(by=['POPULATION'],ascending=False).head(9)
    # Extracts all data from the master dataframe for the above top 10 most populated
    top_zip_subset = df[df['incident_zip'].isin(top_zip_populations['ZIPCODE'])]
    # Top 10 complaint types for 10 most populated NYC-area zipcodes
    s2 = top_zip_subset['complaint_type'].groupby(top_zip_subset['incident_zip']).value_counts()
    top10_by_top_zips = s2.groupby(level=[0]).nlargest(10)
    return top10_by_top_zips

def complaint_index_stats(df):
    #Considering all complaint types. Which boroughs are the biggest "complainers" relative to the size of the population in 2017? 
    #Meaning, calculate a complaint-index that adjusts for population of the borough.
    zip_borough_populations = zip_to_borough_read().merge(us_zip_populations_read(),how='outer',\
    left_on='incident_zip',right_on='ZIPCODE')
    borough_populations = zip_borough_populations.groupby('BOROUGH')['POPULATION'].agg(['sum']).reset_index()
    borough_complaint_totals = df.groupby('BOROUGH').size().reset_index(name='COMPLAINTS')
    population_complaint_totals = borough_populations.merge(borough_complaint_totals,on='BOROUGH')
    population_complaint_totals['complaint_magnitude'] = df6['COMPLAINTS']/df6['sum']
    return population_complaint_totals

df = cleaned_data(raw_data_local())
print('donezo')

donezo


In [83]:
s1 = df['complaint_type'].groupby(df['BOROUGH']).value_counts()
top10_by_borough = s1.groupby('BOROUGH').nlargest(10).to_frame()
top10_by_borough



complaint_type
BOROUGH       BOROUGH       complaint_type                                     
BRONX         BRONX         HEAT/HOT WATER                                69085
                            Noise - Residential                           57928
                            UNSANITARY CONDITION                          24730
                            Blocked Driveway                              24632
                            PAINT/PLASTER                                 19712
                            PLUMBING                                      16581
                            Illegal Parking                               16244
                            Noise - Street/Sidewalk                       14109
                            DOOR/WINDOW                                   11914
                            Street Condition                              11181
BROOKLYN      BROOKLYN      Noise - Residential                           67677
                            HEAT/HOT WATER                                66977
                            Illegal Parking                               55475
                            Blocked Driveway                              49390
                            UNSANITARY CONDITION                          26654
                            Street Condition                              24889
                            Noise - Street/Sidewalk                       21321
                            Water System                                  19511
                            PAINT/PLASTER                                 19398
                            Request Large Bulky Item Collection           16793
MANHATTAN     MANHATTAN     Noise - Residential                           50228
                            HEAT/HOT WATER                                45535
                            Noise - Street/Sidewalk                       28772
                            Noise                                         27924
                            Illegal Parking                               19112
                            Noise - Commercial                            18015
                            Homeless Person Assistance                    16703
                            UNSANITARY CONDITION                          14187
                            Street Condition                              13670
                            PAINT/PLASTER                                 11165
QUEENS        QUEENS        Blocked Driveway                              54251
                            Noise - Residential                           46348
                            Illegal Parking                               45908
                            Street Condition                              29410
                            HEAT/HOT WATER                                29171
                            Request Large Bulky Item Collection           21354
                            Water System                                  18234
                            Street Light Condition                        18230
                            Derelict Vehicle                              16066
                            Building/Use                                  13455
STATEN ISLAND STATEN ISLAND Electronics Waste                             12292
                            Street Condition                              10351
                            Illegal Parking                                8060
                            Noise - Residential                            7037
                            Street Light Condition                         6658
                            Missed Collection (All Materials)              5565
                            Water System                                   5273
                            Sewer                                          3907
                            Blocked Driveway                     

In [102]:
top10_by_borough.reset_index(level=-2,drop=True)

,BOROUGH,complaint_type
complaint_type,,
HEAT/HOT WATER,BRONX,69085
Noise - Residential,BRONX,57928
UNSANITARY CONDITION,BRONX,24730
Blocked Driveway,BRONX,24632
PAINT/PLASTER,BRONX,19712
PLUMBING,BRONX,16581
Illegal Parking,BRONX,16244
Noise - Street/Sidewalk,BRONX,14109
DOOR/WINDOW,BRONX,11914
